<a href="https://colab.research.google.com/github/tayfununal/solving_hamiltonian_with_deeplearning/blob/main/stage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np